# Spatially varying fields

There are several different ways how a spatially varying field can be defined. Let us first define a mesh we are going to use to define the fields.

In [1]:
import discretisedfield as df

p1 = (-50, -50, -50)
p2 = (50, 50, 50)
n = (2, 2, 2)
mesh = df.Mesh(p1=p1, p2=p2, n=n)

## Using a Python function

One of the ways how a spatially varying field can be defined is by using a Python function, which can be passed as `value` argument to `discretisedfield.Field`. It should satisfy three main criteria:
1. It takes one argument. `discretisedfield.Field` is going to pass the coordinates of discertisation cells as tuples of length 3 to this argument.
2. Function should be able to return a value for any coordinate in the mesh
3. The value returned must be of the same dimension as the dimension of the field.

Let us assume we want to have a scalar field which has a value 0 for all points with negative $x$ coordinate and value 1 otherwise.

$$
f(x, y, z)=
    \begin{cases}
      0, & \text{if}\ x<0 \\
      1, & \text{otherwise}
    \end{cases}
$$

The Python function is then:

In [2]:
def my_value_function(pos):
    x, y, z = pos
    if x < 0:
        return 0
    else:
        return 1

After defining the value function, we can define the field.

In [3]:
field = df.Field(mesh, dim=1, value=my_value_function)

If we sample the field at a point with negative value of $x$

In [4]:
field((-10, 5, 5))

0.0

If the $x$ coordinate is positive, we get 1.

In [5]:
field((25, -3, 14))

1.0

The array now has different values

In [6]:
field.array

array([[[[0.],
         [0.]],

        [[0.],
         [0.]]],


       [[[1.],
         [1.]],

        [[1.],
         [1.]]]])

### Value property

It is not very informative to look at `discretisedfield.Field.array` to understand what is the actual value of the field. Therefore, if a unique representation value exists, `discretisedfield.Field.value` is going to return it. For instance:

In [7]:
field.value

<function __main__.my_value_function(pos)>

The source code of this function can be seen as

In [8]:
import inspect
print(inspect.getsource(field.value))

def my_value_function(pos):
    x, y, z = pos
    if x < 0:
        return 0
    else:
        return 1



Now, if we change the value of the field as

In [9]:
field.value = 5

the value of the field is changed

In [10]:
field.value

5

as well as the underlying array

In [11]:
field.array

array([[[[5.],
         [5.]],

        [[5.],
         [5.]]],


       [[[5.],
         [5.]],

        [[5.],
         [5.]]]])

If we violently change the value of a single discretisation cell via array

In [12]:
field.array[0, 0, 0, 0] = 1
field.array

array([[[[1.],
         [5.]],

        [[5.],
         [5.]]],


       [[[5.],
         [5.]],

        [[5.],
         [5.]]]])

no unique representation exists and `field.value` returns an array.

In [13]:
field.value

array([[[[1.],
         [5.]],

        [[5.],
         [5.]]],


       [[[5.],
         [5.]],

        [[5.],
         [5.]]]])

Similar to scalar fields, a Python function can be used to set the value of a vector field. This time, the function should return three-dimensional values.

In [14]:
def vector_value_function(pos):
    x, y, z = pos
    vx = x
    vy = x*y
    vz = x*y*z
    
    return (vx, vy, vz)

This function can now be used at the definition of the field:

In [15]:
field = df.Field(mesh, dim=3, value=vector_value_function)

Its value is now:

In [16]:
field.value

<function __main__.vector_value_function(pos)>

In [17]:
field.array

array([[[[   -25.,    625., -15625.],
         [   -25.,    625.,  15625.]],

        [[   -25.,   -625.,  15625.],
         [   -25.,   -625., -15625.]]],


       [[[    25.,   -625.,  15625.],
         [    25.,   -625., -15625.]],

        [[    25.,    625., -15625.],
         [    25.,    625.,  15625.]]]])

## Using mesh regions

If regions were defined as a part of the mesh, and we want to set the value of the field differently in those regions, we can employ some of the functionality of regions. Let us assume that in the mesh we defined we want to have two regions. Region 1 is going to include all cells with negative $y$ coordinate and region 2 cells with positive $y$ coordinate. Our mesh would be:

In [18]:
subregions = {'region1': df.Region(p1=(-50, -50, -50), p2=(50, 0, 50)),
              'region2': df.Region(p1=(-50, 0, -50), p2=(50, 50, 50))}
mesh = df.Mesh(p1=p1, p2=p2, n=n, subregions=subregions)

Python function employing these regions can now be

In [19]:
def regions_function(pos):
    if pos in mesh.subregions['region1']:
        return (1, 0, 0)
    elif pos in mesh.subregions['region2']:
        return (0, 1, 0)
    else:
        return (0, 0, 0)

We can now pass this function to the `discretisedfield.Field` class

In [20]:
field = df.Field(mesh, dim=3, value=regions_function)

For a negative value of $y$, we get:

In [21]:
field((10, -10, 10))

(1.0, 0.0, 0.0)

And for positive:

In [22]:
field((10, 30, 10))

(0.0, 1.0, 0.0)

Another way of setting the field is passing the dictionary as a value to the field. However, there are several warnings that must be taken care of:
1. Region names must be the same as defined regions in `discretisedfield.Mesh`.
2. Only those points in the mesh which belong to one of the regions will be set. If there is a point which is not in any of the regions, its value is set to zero.

In [23]:
region_values = {'region1': (1, 1, 1), 'region2': (2, 2, 2)}
field.value = region_values

Now, we can sample points in two regions.

In [24]:
field((-10, -10, -10))

(1.0, 1.0, 1.0)

In [25]:
field((10, 10, 10))

(2.0, 2.0, 2.0)

Initialisation can be simplified if several subregions have the same value or only parts of the region are contained within one of the subregions. It is possible to omit any number of subregion keys and specify the special key ``default``. All points not contained in one of the explicitely given subregions are then set to the value of ``default``.

In [26]:
region_values = {'region1': (0, 1, 1), 'default': (2, 2, 0)}
field.value = region_values

In [27]:
field((-10, -10, -10))

(0.0, 1.0, 1.0)

In [28]:
field((10, 10, 10))

(2.0, 2.0, 0.0)

In [29]:
region_values = {'default': (2, 2, 1)}
field.value = region_values

In [30]:
field((-10, -10, -10))

(2.0, 2.0, 1.0)

In [31]:
field((10, 10, 10))

(2.0, 2.0, 1.0)

## Using another Field object

Sometimes it is necessary to "resample" the field using a different mesh. Another field can be passed as a value to the new field. If our new mesh is:

In [32]:
p1 = (-10, -10, -10)
p2 = (10, 10, 10)
cell = (5, 5, 5)
new_mesh = df.Mesh(p1=p1, p2=p2, cell=cell)

The field we initialised previouly has the value

In [33]:
field.array

array([[[[2., 2., 1.],
         [2., 2., 1.]],

        [[2., 2., 1.],
         [2., 2., 1.]]],


       [[[2., 2., 1.],
         [2., 2., 1.]],

        [[2., 2., 1.],
         [2., 2., 1.]]]])

We can now resample that field as

In [34]:
new_field = df.Field(new_mesh, dim=3, value=field)

The values are now

In [35]:
new_field.array.shape

(4, 4, 4, 3)

In [36]:
new_field((-5, -5, -5))

(2.0, 2.0, 1.0)

In [37]:
new_field((5, 5, 5))

(2.0, 2.0, 1.0)

## Other

Full description of all existing functionality can be found in the [API Reference](https://discretisedfield.readthedocs.io/en/latest/_autosummary/discretisedfield.Field.html).